<a href="https://colab.research.google.com/github/mukemitt/ai-agent/blob/main/ai-agent-with-openais-assistants-api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>